In [8]:
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go

from joblib import Memory
memory = Memory("cache")

In [9]:
@memory.cache
def geopos_from_zipcode(country_code, zip_code):
  API_KEY = "API_KEY"
  URL = f"https://api.openweathermap.org/geo/1.0/zip?zip={zip_code},{country_code}&appid={API_KEY}"
  response =  requests.get(URL)
  json = response.json()
  if "lat" in json and "lon" in json:
    return json["lat"], json["lon"]
  else:
    return None, None

In [10]:
# Import Meteostat library and dependencies
from datetime import datetime
import plotly.express as px
from meteostat import Point, Hourly

In [11]:
ccodes = pd.read_csv("countries.csv", sep=" \t")

In [12]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)

app.layout = html.Div([
  html.Label("Country Code"),
  dcc.Dropdown(id="country-dropdown", 
               options=ccodes["A2"]
               #options=[{"label": row["Country"], "value": row["A2"]} for _, row in ccodes.iterrows()]
               ),
  html.Label("Zip Code"),
  dcc.Input(id="zip-input", type="number", placeholder="Enter a Zip Code", debounce=0.5),

  dcc.Graph(id="weather-graph")
])

@memory.cache
def fetch_temperatures(country_code, zip_code, start, end):
  lat, lon = geopos_from_zipcode(country_code, zip_code)
  if lat is None or lon is None:
    return pd.DataFrame(columns=["temp"])
  
  location = Point(lat, lon, 500)
  data = Hourly(location, start, end)
  data = data.fetch()
  return data

@app.callback(
  Output("weather-graph", "figure"),
  Input("country-dropdown", "value"),
  Input("zip-input", "value")
)
def update_graph(country_code, zip_code):
  if country_code is None or zip_code is None:
    return {}
  
  # Set time period
  start = datetime(2021, 1, 1)
  end = datetime(2022, 12, 31)
  
  data = fetch_temperatures(country_code, zip_code, start, end)
  return px.line(data, y='temp')


In [13]:
app.run_server()

/home/missinguser/CSE/fa2023/venv/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

